# Linkedin

**Trabajos.**

https://www.linkedin.com/jobs/search/?geoId=105646813&keywords=data%20science&location=Espa%C3%B1a

In [ ]:
codes={'mexico':106262181, 'españa':105646813, 'usa':103644278,
       'brasil':106057199, 'portugal':100364837, 'alemania': 101282230,
       'francia':105015875, 'holanda':102890719}

In [ ]:
countries=['mexico', 'españa', 'usa', 'brasil', 'portugal', 'alemania', 'francia', 'holanda']
           

keyword=['web developer', 'javascript  developer', 'back-end  developer', 'front-end  developer',
         'software engineer', 'frontend developer', 'backend developer',
         'web designer', 'ux designer', 'web graphic designer',
         'ux/ui designer', 'product designer',
         'data engineer', 'data analyst','data scientist', 'database administrator']     

**Salarios.**

https://www.linkedin.com/salary/explorer?countryCode=es&geoId=105646813&titleId=340

In [ ]:
codes={'mx':103323778, 'es':105646813, 'us':103644278,
       'br':106057199, 'de': 101282230,
       'fr':105015875, 'nl':102890719}

## Plots